In [1]:
import pandas as pd
import jinja2
from IPython.display import display, display_pretty, Javascript, HTML

In [2]:
pd_input_matrix = pd.read_csv("data/test.no_sort_vector.matrix.txt",delim_whitespace=True)
test_matrix = pd_input_matrix.to_json(orient="split")

In [3]:
import json
with open("data/test.no_sort_vector.dendrogram.json") as f:
    test_dendrogram = json.load(f)

In [4]:
display(HTML(
"""
<head>
    <script src="//d3js.org/d3.v3.min.js" charset="utf-8"></script>
</head>
<style>
body {
  font: 10px sans-serif;
}
.chart{
    width: 100%;
    height: 600px;
    left: 0;
    top: 0;
    position: relative;
}
.dendrogram {
    //float: left;
    width: 20%;
    height: 60%;
    position: absolute;
    bottom: 0;
    left: 0;
    //margin-left: 5%;
    //margin-right: 5%;
    //padding-left: 2%;
    //padding-right: 2%;
    //background-color: cyan;
}
.vert_dendrogram {
    width: 60%;
    height: 20%;
    position: absolute;
    bottom: 80%;
    left: 40%;
    //background-color: cyan;
}
.heatmap {
    //float: right;
    width: 60%;
    height: 60%;
    position: absolute;
    bottom: 0;
    left: 40%;
    //margin-left: 5%;
    //margin-right: 5%;
    //background-color: black;
}
.row_names {
    width: 18%;
    height: 60%;
    position: absolute;
    bottom: 0;
    left: 21%;
    //padding-left: 1%;
    //padding-right: 1%;
    //box-sizing: border-box;
    overflow: hidden;
    //overflow-y: hidden;
    //background-color: magenta;
}
.vert_row_names {
    width: 60%;
    height: 18%;
    position: absolute;
    top: 21%;
    left: 40%;
    overflow: hidden;
    //padding-top: 1%;
    //padding-bottom: 1%;
    //box-sizing: border-box;
    //background-color: magenta;
}
/*
.heatmap_header {
    width: 60%;
    height: 5%;
    position: absolute;
    bottom: 82%;
    left: 40%;
    overflow: visible;
    //background-color: yellow;
}
*/
.legend {
    width: 20%;
    height: 5%;
    position: absolute;
    top: 28%;
    left: 10%;
    overflow: visible;
}
.tooltip {
    visibility:hidden;
    background: lightblue;
    border:solid gray;
    position: absolute;
}
</style>
<body>
<div class="chart" id="chart">
    <div class="dendrogram" id="dendrogram"/>
    <div class="heatmap" id="heatmap"/>
    <div class="row_names" id="row_names"/>
    <div class="legend" id="legend"/>
    <div class="vert_dendrogram" id="vert_dendrogram"/>
    <div class="vert_row_names" id="vert_row_names"/>
</div>
<div class="tooltip"/>
</body>
"""
))

test_template = jinja2.Template(
"""
function writeDendrogram() {
    let line_coords = [];

    let x_max = parseFloat(Math.max(...[].concat.apply([],dendrogram_data["dcoord"])));
    let y_max = parseFloat(Math.max(...[].concat.apply([],dendrogram_data["icoord"])));

    let x_min = parseFloat(Math.min(...[].concat.apply([],dendrogram_data["dcoord"])));
    let y_min = parseFloat(Math.min(...[].concat.apply([],dendrogram_data["icoord"])));

    let dend_height = $("#chart>#dendrogram").innerHeight();
    let dend_width = $("#chart>#dendrogram").innerWidth();

    let leaf_num = dendrogram_data["leaves"].length;

    for (let i in dendrogram_data["dcoord"]) {
        for (let j in [0, 1, 2]) {
            line_coords.push({
                y1: ((0.5/leaf_num)*dend_height)+((parseFloat(dendrogram_data["icoord"][i][j])-y_min)/(y_max-y_min))*(dend_height*((leaf_num-1)/leaf_num)),
                y2: ((0.5/leaf_num)*dend_height)+((parseFloat(dendrogram_data["icoord"][i][parseInt(j)+1])-y_min)/(y_max-y_min))*(dend_height*((leaf_num-1)/leaf_num)),
                x1: dend_width-((parseFloat(dendrogram_data["dcoord"][i][j])-x_min)/(x_max-x_min))*dend_width,
                x2: dend_width-((parseFloat(dendrogram_data["dcoord"][i][parseInt(j)+1])-x_min)/(x_max-x_min))*dend_width
            });
        }
    }

    let svg = d3.select("#chart>#dendrogram")
        .append("svg")
        .attr("width", dend_width)
        .attr("height", dend_height)
        .style("overflow", "visible");

    svg.append("g")
        .selectAll("line")                 
        .data(line_coords)
        .enter()
        .append("line") 
        .attr("x1", function(d) { return d.x1; })
        .attr("x2", function(d) { return d.x2; })
        .attr("y1", function(d) { return d.y1; })
        .attr("y2", function(d) { return d.y2; })
        .style("stroke", "black")
        .style("stroke-width", 1);
}

function writeHeatmap() {
    let heatmap_height = $("#chart>#heatmap").innerHeight(),
        heatmap_width = $("#chart>#heatmap").innerWidth();

    let heatmap_column_number = heatmap_data["columns"].length,
        heatmap_row_number = heatmap_data["index"].length;

    let heatmap_cell_height = heatmap_height/heatmap_row_number,
        heatmap_cell_width = heatmap_width/heatmap_column_number;

    let colorScale = d3.scale.linear()
        .domain([-1, 0, 1])
        .range(['blue','white','red']);

    let tooltip = d3.select("body")
        .append("div")
        .style("position", "absolute")
        .style("z-index", "10")
        .style("visibility", "hidden")
        .text("a simple tooltip");

    let svg = d3.select("#chart>#heatmap")
        .append("svg")
        .attr("width", heatmap_width)
        .attr("height", heatmap_height);

    svg.append("g")
        .selectAll("g")                 
        .data(heatmap_data["data"])
        .enter()
        .append("g") 
        .selectAll("rect") 
        .data( function(d,i,j) { return d; } ) 
        .enter() 
        .append("rect")
        .text( function(d,i,j) { return d; } )
        .attr("x", function(d,i,j) { return (i * heatmap_cell_width); })
        .attr("y", function(d,i,j) { return (j * heatmap_cell_height); })
        .attr("width", function(d) { return heatmap_cell_width; })
        .attr("height", function(d) { return heatmap_cell_height; })
        .style("fill", function(d) { return colorScale(d); })
        .on("mouseover", function () {
            d3.select(this).style("stroke", "black").style("stroke-width", "1");
            return tooltip.style("visibility", "visible");
        })
        .on("mousemove", function (d, i, j) {
            //console.log(d3.event);
            return tooltip
                .style("top", (event.pageY + 16) + "px")
                .style("left", (event.pageX + 16) + "px")
                .html(heatmap_data["index"][j] + "<br/>" + heatmap_data["columns"][i] + "<br/>" + d);
        })
        .on("mouseout", function () {
            d3.select(this).style("stroke", "none");
            return tooltip.style("visibility", "hidden");
        });

}

function writeRowNames() {
    let div_height = $("#chart>#row_names").innerHeight(),
        div_width = $("#chart>#row_names").innerWidth();

    let row_number = heatmap_data["index"].length;

    //let spacing = div_width * 0.1;

    let svg = d3.select("#chart>#row_names")
        .append("svg")
        .attr("width", div_width)
        .attr("height", div_height)
        //.attr("x", div_width*0.1)
        //.attr("transform", "translate(" + div_width*0.1 + ")")
        //.style("overflow", "scroll");

    svg.append("g")
        //.attr("transform", "translate(" + div_width*0.05 + ")")
        .selectAll("text")                 
        .data(heatmap_data["index"])
        .enter()
        .append("text")
        .text(function(d,i) { return d;})
        .attr("x", 0)
        .attr("y", function(d,i) {return (((0.5/row_number)*div_height) + i*(div_height/row_number)); })
        .attr("font-family", "sans-serif")
        .attr("font-size", "12px")
        .attr("fill", "black")
        .style("dominant-baseline", "middle");
}

/*
function writeHeatmapHeader() {
    let div_height = $("#chart>#heatmap_header").innerHeight(),
        div_width = $("#chart>#heatmap_header").innerWidth();

    let range_start = heatmap_data["columns"][0].split(":")[0],
        range_end = heatmap_data["columns"][heatmap_data["columns"].length-1].split(":")[1];

    //console.log(range_start);
    //console.log(range_end);

    let zero_position = div_width/(range_end-range_start)*(0-range_start);

    //console.log(div_width);
    //console.log(zero_position);

    let svg = d3.select("#chart>#heatmap_header")
        .append("svg")
        .attr("width", div_width)
        .attr("height", div_height)
        .style("overflow", "visible");

    let header_lines = [
        {text: range_start, position: 0},
        {text: range_end, position: div_width},
        {text: "0", position: zero_position}
    ];

    svg.append("g")
        .selectAll("line")                 
        .data(header_lines)
        .enter()
        .append("line")
        .attr("x1", function(d) {return d.position;})
        .attr("x2", function(d) {return d.position;})
        .attr("y1", 0.5*div_height)
        .attr("y2", div_height)
        .style("stroke", "black")
        .style("stroke-width", 1);

    svg.append("line")
        .attr("x1", 0)
        .attr("x2", div_width)
        .attr("y1", div_height)
        .attr("y2", div_height)
        .style("stroke", "black")
        .style("stroke-width", 1);

    svg.append("g")
        .selectAll("text")                 
        .data(header_lines)
        .enter()
        .append("text")
        .text(function(d) { return d.text;})
        .attr("x", function(d) {return d.position;})
        .attr("y", 0.45*div_height)
        .attr("font-family", "sans-serif")
        .attr("font-size", "12px")
        .attr("fill", "black")
        .style("text-anchor", "middle");
}
*/

function writeLegend() {
    let div_height = $("#chart>#legend").innerHeight(),
        div_width = $("#chart>#legend").innerWidth();

    let svg = d3.select("#chart>#legend")
        .append("svg")
        .attr("width", div_width)
        .attr("height", div_height)
        .style("overflow", "visible");

    let gradient = svg
        .append("linearGradient")
        .attr("y1", "0")
        .attr("y2", "0")
        .attr("x1", "0")
        .attr("x2", div_width)
        .attr("id", "gradient")
        .attr("gradientUnits", "userSpaceOnUse")

    gradient
        .append("stop")
        .attr("offset", "0")
        .attr("stop-color", "blue")

    gradient
        .append("stop")
        .attr("offset", "0.5")
        .attr("stop-color", "white")

    gradient
        .append("stop")
        .attr("offset", "1")
        .attr("stop-color", "red")

    svg.append("rect")
        .attr("width", div_width)
        .attr("height", 0.5*div_height)
        .attr("x", "0")
        .attr("y", 0.5*div_height)
        .attr("fill", "url(#gradient)")
        .attr("stroke", "black")
        .attr("stroke-width", "1");

    let legend_lines = [
        {text: "-1", position: 0},
        {text: "0", position: 0.5*div_width},
        {text: "1", position: div_width}
    ];

    svg.append("g")
        .selectAll("line")                 
        .data(legend_lines)
        .enter()
        .append("line")
        .attr("x1", function(d) {return d.position;})
        .attr("x2", function(d) {return d.position;})
        .attr("y1", 0.3*div_height)
        .attr("y2", 0.5*div_height)
        .style("stroke", "black")
        .style("stroke-width", 1);

    svg.append("g")
        .selectAll("text")                 
        .data(legend_lines)
        .enter()
        .append("text")
        .text(function(d) { return d.text;})
        .attr("x", function(d) {return d.position;})
        .attr("y", 0.25*div_height)
        .attr("font-family", "sans-serif")
        .attr("font-size", "12px")
        .attr("fill", "black")
        .style("text-anchor", "middle");
}

function writeVertDendrogram() {
    let line_coords = [];

    let y_max = parseFloat(Math.max(...[].concat.apply([],dendrogram_data["dcoord"])));
    let x_max = parseFloat(Math.max(...[].concat.apply([],dendrogram_data["icoord"])));

    let y_min = parseFloat(Math.min(...[].concat.apply([],dendrogram_data["dcoord"])));
    let x_min = parseFloat(Math.min(...[].concat.apply([],dendrogram_data["icoord"])));

    let dend_height = $("#chart>#vert_dendrogram").innerHeight();
    let dend_width = $("#chart>#vert_dendrogram").innerWidth();

    let leaf_num = dendrogram_data["leaves"].length;

    for (let i in dendrogram_data["dcoord"]) {
        for (let j in [0, 1, 2]) {
            line_coords.push({
                y1: dend_height-((parseFloat(dendrogram_data["dcoord"][i][j])-y_min)/(y_max-y_min))*(dend_height),
                y2: dend_height-((parseFloat(dendrogram_data["dcoord"][i][parseInt(j)+1])-y_min)/(y_max-y_min))*(dend_height),
                x1: ((0.5/leaf_num)*dend_width)+((parseFloat(dendrogram_data["icoord"][i][j])-x_min)/(x_max-x_min))*(dend_width*((leaf_num-1)/leaf_num)),
                x2: ((0.5/leaf_num)*dend_width)+((parseFloat(dendrogram_data["icoord"][i][parseInt(j)+1])-x_min)/(x_max-x_min))*(dend_width*((leaf_num-1)/leaf_num))
            });
        }
    }

    let svg = d3.select("#chart>#vert_dendrogram")
        .append("svg")
        .attr("width", dend_width)
        .attr("height", dend_height)
        .style("overflow", "visible");

    svg.append("g")
        .selectAll("line")                 
        .data(line_coords)
        .enter()
        .append("line") 
        .attr("x1", function(d) { return d.x1; })
        .attr("x2", function(d) { return d.x2; })
        .attr("y1", function(d) { return d.y1; })
        .attr("y2", function(d) { return d.y2; })
        .style("stroke", "black")
        .style("stroke-width", 1);
}

function writeVertRowNames() {

    let div_height = $("#chart>#vert_row_names").innerHeight(),
        div_width = $("#chart>#vert_row_names").innerWidth();

    let row_number = heatmap_data["index"].length;

    //let spacing = div_width * 0.1;

    let svg = d3.select("#chart>#vert_row_names")
        .append("svg")
        .attr("width", div_width)
        .attr("height", div_height)
        //.attr("x", div_width*0.1)
        //.attr("transform", "translate(" + div_width*0.1 + ")")
        //.style("overflow", "scroll");

    svg.append("g")
        //.attr("transform", "translate(0," + div_height*0.05 + ")")
        .selectAll("text")                 
        .data(heatmap_data["index"])
        .enter()
        .append("text")
        .text(function(d,i) { return d;})
        .attr("x", function(d,i) {return (((0.5/row_number)*div_width) + i*(div_width/row_number)); })
        .attr("y", 0)
        .attr("font-family", "sans-serif")
        .attr("font-size", "12px")
        .attr("fill", "black")
        .attr("transform", function(d,i) {return "rotate(90 " + (((0.5/row_number)*div_width) + i*(div_width/row_number)) + "," + 0 +")";})
        .style("dominant-baseline", "middle");
}

var dendrogram_data = {{dendrogram_data}};
var heatmap_data = {{heatmap_data}};

writeDendrogram();
writeVertDendrogram();
writeHeatmap();
writeRowNames();
writeVertRowNames();
//writeHeatmapHeader();
writeLegend();
"""
)
display(Javascript(test_template.render(dendrogram_data=test_dendrogram, heatmap_data=test_matrix)))

<IPython.core.display.Javascript object>